This notebook fills the NaN values of isochrones, routes, poi features with nearest neighbour values due to time constraints.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Initialize Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Rental Data Imputation") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

print(f"Spark version: {spark.version}")
print(f"Spark UI available at: {spark.sparkContext.uiWebUrl}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/12 08:37:10 WARN Utils: Your hostname, Pikachu.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.46 instead (on interface en0)
25/10/12 08:37:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/12 08:37:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 4.0.1
Spark UI available at: http://192.168.1.46:4040


In [2]:
import pandas as pd
import geopandas as gpd
from utils.preprocess import PreprocessUtils
from shapely import wkt

# Initialize the preprocessor
preprocessor = PreprocessUtils()

# ORS (Open Route Service) PTV Routes

In [3]:
# read in everything we obtained thus far which is in data/processed/routes/ptv/routes_ptv_combined.csv 

routes_ptv = pd.read_csv("../data/processed/routes/ptv/routes_ptv_combined.csv")

# lower case column names and snake case
routes_ptv.columns = routes_ptv.columns.str.lower().str.replace(" ", "_")

# drop the rows where Station_ID is NaN
routes_ptv = routes_ptv[routes_ptv["stationid"].notna()]

# drop the rows where Station_ID is 0
routes_ptv = routes_ptv[routes_ptv["stationid"] != 0]

# drop coordinates column from routes_ptv to prepare for merge
routes_ptv = routes_ptv.drop(columns=["coordinates"])



In [4]:
routes_ptv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5760 entries, 0 to 11491
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   property_id       5760 non-null   int64  
 1   stationid         5760 non-null   object 
 2   min_route_dist_m  5760 non-null   float64
 3   min_route_dur_s   5760 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 225.0+ KB


In [5]:
# read in cleaned_listings_sampled.csv which is in data/processed/domain/cleaned_listings_sampled.csv

cleaned_listings_sampled = pd.read_csv("../data/processed/domain/cleaned_listings_sampled.csv")
# convert property_id to int64
cleaned_listings_sampled["property_id"] = cleaned_listings_sampled["property_id"].astype("Int64")

# Convert string coordinates to actual geometry objects
cleaned_listings_sampled["coordinates"] = cleaned_listings_sampled["coordinates"].apply(wkt.loads)

# Convert to GeoDataFrame
cleaned_listings_sampled = gpd.GeoDataFrame(cleaned_listings_sampled, geometry="coordinates")

# merge cleaned_listings_sampled with routes_ptv on property_id
cleaned_listings_sampled = cleaned_listings_sampled.merge(routes_ptv, on="property_id", how="left")

# rename stationid to closest_ptv_station_id
cleaned_listings_sampled = cleaned_listings_sampled.rename(columns={"stationid": "closest_ptv_station_id"})

cleaned_listings_sampled.info()



<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13101 entries, 0 to 13100
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   property_id             13101 non-null  Int64   
 1   rental_price            13101 non-null  float64 
 2   bedrooms                13101 non-null  int64   
 3   bathrooms               13101 non-null  int64   
 4   car_spaces              13101 non-null  int64   
 5   property_type           13101 non-null  object  
 6   suburb                  13101 non-null  object  
 7   year                    13101 non-null  int64   
 8   quarter                 13101 non-null  int64   
 9   longitude               13101 non-null  float64 
 10  latitude                13101 non-null  float64 
 11  coordinates             13101 non-null  geometry
 12  closest_ptv_station_id  2768 non-null   object  
 13  min_route_dist_m        2768 non-null   float64 
 14  min_route_dur_

In [8]:
missing_routes = cleaned_listings_sampled[cleaned_listings_sampled["closest_ptv_station_id"].isnull()][['property_id', 'coordinates']]

In [ ]:
batch_size = 500
preprocessor.split_into_batches(missing_routes, batch_size, "../data/raw/missing_routes")

TypeError: PreprocessUtils.split_into_batches() got an unexpected keyword argument 'verbose'

In [22]:
# impute by nearest neighbour
cleaned_listings_sampled['closest_ptv_station_id_imputed'] = preprocessor.impute_by_nearest_point(cleaned_listings_sampled, 'closest_ptv_station_id')
cleaned_listings_sampled['min_route_dist_m_imputed'] = preprocessor.impute_by_nearest_point(cleaned_listings_sampled, 'min_route_dist_m')
cleaned_listings_sampled['min_route_dur_s_imputed'] = preprocessor.impute_by_nearest_point(cleaned_listings_sampled, 'min_route_dur_s')



Imputing null values in 1 column(s): ['closest_ptv_station_id']
Total rows with null values to impute: 10333
Successfully imputed: 10333 rows, Could not impute (no data available): 0
Imputing null values in 1 column(s): ['min_route_dist_m']
Total rows with null values to impute: 10333
Successfully imputed: 10333 rows, Could not impute (no data available): 0
Imputing null values in 1 column(s): ['min_route_dur_s']
Total rows with null values to impute: 10333
Successfully imputed: 10333 rows, Could not impute (no data available): 0


In [23]:
cleaned_listings_sampled.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 13101 entries, 0 to 13100
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   property_id                     13101 non-null  Int64   
 1   rental_price                    13101 non-null  float64 
 2   bedrooms                        13101 non-null  int64   
 3   bathrooms                       13101 non-null  int64   
 4   car_spaces                      13101 non-null  int64   
 5   property_type                   13101 non-null  object  
 6   suburb                          13101 non-null  object  
 7   year                            13101 non-null  int64   
 8   quarter                         13101 non-null  int64   
 9   longitude                       13101 non-null  float64 
 10  latitude                        13101 non-null  float64 
 11  coordinates                     13101 non-null  geometry
 12  closest_pt

# OSM (Open Street Map) POI

In [24]:
# read in poi_features_combined.csv which is in data/processed/poi_features/poi_features_combined.csv
poi_features = pd.read_csv("../data/processed/poi_features/poi_features_combined_before_imputation.csv")

# converrt column name of PropertyID to property_id
poi_features = poi_features.rename(columns={"PropertyID": "property_id"})


In [25]:
# show null value information
poi_features.isnull().sum()



property_id                                0
count_atm                               2102
count_bank                              1749
count_childcare                         1707
count_clinic                            1404
count_community_centre                  1308
count_doctors                           1706
count_fast_food                          641
count_fuel                               607
count_kindergarten                       980
count_parking_space                     1500
count_restaurant                         921
min_dist_atm                            2102
min_dist_bank                           1749
min_dist_childcare                      1707
min_dist_clinic                         1404
min_dist_community_centre               1308
min_dist_doctors                        1706
min_dist_fast_food                       641
min_dist_fuel                            607
min_dist_kindergarten                    980
min_dist_parking_space                  1500
min_dist_r

In [26]:
# merge poi_features with cleaned_listings_sampled on property_id
cleaned_listings_sampled = cleaned_listings_sampled.merge(poi_features, on="property_id", how="left")

# show null value information
cleaned_listings_sampled.isnull().sum()

property_id                                 0
rental_price                                0
bedrooms                                    0
bathrooms                                   0
car_spaces                                  0
property_type                               0
suburb                                      0
year                                        0
quarter                                     0
longitude                                   0
latitude                                    0
coordinates                                 0
closest_ptv_station_id                  10333
min_route_dist_m                        10333
min_route_dur_s                         10333
closest_ptv_station_id_imputed              0
min_route_dist_m_imputed                    0
min_route_dur_s_imputed                     0
count_atm                               11636
count_bank                              11472
count_childcare                         11449
count_clinic                      

In [27]:
# get the column names of cleaned_listings_sampled with null values
columns_with_na = list(cleaned_listings_sampled.columns[cleaned_listings_sampled.isnull().any()])


In [ ]:
for col in columns_with_na:
    cleaned_listings_sampled[col+"_imputed"] = preprocessor.impute_by_nearest_point(cleaned_listings_sampled, col)

# Verify that all null values have been imputed
print("\nNull values remaining:")
print(cleaned_listings_sampled[columns_with_na].isnull().sum())



Imputing null values in 1 column(s): ['closest_ptv_station_id']
Total rows with null values to impute: 10333


Successfully imputed: 10333 rows, Could not impute (no data available): 0
Imputing null values in 1 column(s): ['min_route_dist_m']
Total rows with null values to impute: 10333
Successfully imputed: 10333 rows, Could not impute (no data available): 0
Imputing null values in 1 column(s): ['min_route_dur_s']
Total rows with null values to impute: 10333
Successfully imputed: 10333 rows, Could not impute (no data available): 0
Imputing null values in 1 column(s): ['count_atm']
Total rows with null values to impute: 11636
Successfully imputed: 11636 rows, Could not impute (no data available): 0
Imputing null values in 1 column(s): ['count_bank']
Total rows with null values to impute: 11472
Successfully imputed: 11472 rows, Could not impute (no data available): 0
Imputing null values in 1 column(s): ['count_childcare']
Total rows with null values to impute: 11449
Successfully imputed: 11449 rows, Could not impute (no data available): 0
Imputing null values in 1 column(s): ['count_clinic']
To

# ORS Isochrones

In [ ]:
# read in driving_isochrones_wayback 
driving_isochrones_wayback = preprocessor.merge_batches("../data/processed/isochrones/wayback/driving", verbose=True)
# drop the coordinates column from driving_isochrones_wayback
driving_isochrones_wayback = driving_isochrones_wayback.drop(columns=["coordinates"])
# change the column names of 5min, 10min, 15min to driving_5min, driving_10min, driving_15min
driving_isochrones_wayback.rename(columns={"5min": "driving_5min", "10min": "driving_10min", "15min": "driving_15min"}, inplace=True)

# read in walking_isochrones_wayback
walking_isochrones_wayback = preprocessor.merge_batches("../data/processed/isochrones/wayback/walking", verbose=True)
# drop the coordinates column from walking_isochrones_wayback
walking_isochrones_wayback = walking_isochrones_wayback.drop(columns=["coordinates"])
# change the column names of 5min, 10min, 15min to walking_5min, walking_10min, walking_15min
walking_isochrones_wayback.rename(columns={"5min": "walking_5min", "10min": "walking_10min", "15min": "walking_15min"}, inplace=True)

# merge the driving_isochrones_wayback and walking_isochrones_wayback on property_id
isochrones_wayback = pd.merge(driving_isochrones_wayback, walking_isochrones_wayback, on="property_id", how="outer")


Starting merge process...
Input directory: ../data/processed/isochrones/wayback/driving
File pattern: *.csv
------------------------------------------------------------
Found 13 files to merge:
  - isochrone_0001.csv
  - isochrone_0002.csv
  - isochrone_0003.csv
  - isochrone_0004.csv
  - isochrone_0005.csv
  - isochrone_0006.csv
  - isochrone_0007.csv
  - isochrone_0008.csv
  - isochrone_0009.csv
  - isochrone_0010.csv
  - isochrone_0011.csv
  - isochrone_0012.csv
  - isochrone_0013.csv

  Loaded: isochrone_0001.csv (500 rows)
  Loaded: isochrone_0002.csv (500 rows)
  Loaded: isochrone_0003.csv (500 rows)
  Loaded: isochrone_0004.csv (500 rows)
  Loaded: isochrone_0005.csv (500 rows)
  Loaded: isochrone_0006.csv (500 rows)
  Loaded: isochrone_0007.csv (500 rows)
  Loaded: isochrone_0008.csv (500 rows)
  Loaded: isochrone_0009.csv (500 rows)
  Loaded: isochrone_0010.csv (500 rows)
  Loaded: isochrone_0011.csv (500 rows)
  Loaded: isochrone_0012.csv (500 rows)
  Loaded: isochrone_0013.c

In [ ]:
isochrones_wayback.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   driving_5min   4929 non-null   object 
 1   driving_10min  4929 non-null   object 
 2   driving_15min  4929 non-null   object 
 3   property_id    6499 non-null   float64
 4   walking_5min   5629 non-null   object 
 5   walking_10min  5629 non-null   object 
 6   walking_15min  5629 non-null   object 
dtypes: float64(1), object(6)
memory usage: 355.5+ KB


In [ ]:
live_listings = pd.read_csv("../data/processed/domain/live_listings.csv")

# Convert string coordinates to actual geometry objects
live_listings["coordinates"] = live_listings["coordinates"].apply(wkt.loads)

# Convert to GeoDataFrame
live_listings = gpd.GeoDataFrame(live_listings, geometry="coordinates")

live_listings.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 12649 entries, 0 to 12648
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   property_id         12649 non-null  float64 
 1   rental_price        12649 non-null  float64 
 2   suburb              12649 non-null  object  
 3   postcode            12649 non-null  int64   
 4   property_type       12649 non-null  object  
 5   year                12649 non-null  int64   
 6   quarter             12649 non-null  int64   
 7   bedrooms            12649 non-null  int64   
 8   bathrooms           12649 non-null  int64   
 9   car_spaces          12649 non-null  int64   
 10  age_0_to_19         12649 non-null  float64 
 11  age_20_to_39        12649 non-null  float64 
 12  age_40_to_59        12649 non-null  float64 
 13  age_60_plus         12649 non-null  float64 
 14  agency_name         12649 non-null  object  
 15  appointment_only    12649 no

In [ ]:
# read in isochrones_combined.csv which is in data/processed/isochrones/isochrones_combined.csv
isochrones_driving = pd.read_csv("../data/processed/isochrones/driving/isochrones_driving_combined.csv")
# rename 5min, 10min, 15min to driving_5min, driving_10min, driving_15min   
isochrones_driving.rename(columns={"5min": "driving_5min", "10min": "driving_10min", "15min": "driving_15min"}, inplace=True)
# drop the coordinates column from isochrones_driving
isochrones_driving = isochrones_driving.drop(columns=["coordinates"])

isochrones_walking = pd.read_csv("../data/processed/isochrones/walking/isochrones_walking_combined.csv")
# rename 5min, 10min, 15min to walking_5min, walking_10min, walking_15min
isochrones_walking.rename(columns={"5min": "walking_5min", "10min": "walking_10min", "15min": "walking_15min"}, inplace=True)
# drop the coordinates column from isochrones_walking
isochrones_walking = isochrones_walking.drop(columns=["coordinates"])

# merge isochrones_driving and isochrones_walking on property_id
isochrones_live = pd.merge(isochrones_driving, isochrones_walking, on="property_id", how="outer")

# select only rows where property_id is in live_listings
isochrones_live = isochrones_live[isochrones_live["property_id"].isin(live_listings["property_id"])]

isochrones_live.info()




<class 'pandas.core.frame.DataFrame'>
Index: 15005 entries, 2 to 34913
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   driving_5min   13863 non-null  object 
 1   driving_10min  13863 non-null  object 
 2   driving_15min  13863 non-null  object 
 3   property_id    15005 non-null  float64
 4   walking_5min   9695 non-null   object 
 5   walking_10min  9695 non-null   object 
 6   walking_15min  9695 non-null   object 
dtypes: float64(1), object(6)
memory usage: 937.8+ KB


In [ ]:
# stack isochrones_live and isochrones_wayback using concat
isochrones = pd.concat([isochrones_live, isochrones_wayback])

isochrones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21504 entries, 2 to 6498
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   driving_5min   18792 non-null  object 
 1   driving_10min  18792 non-null  object 
 2   driving_15min  18792 non-null  object 
 3   property_id    21504 non-null  float64
 4   walking_5min   15324 non-null  object 
 5   walking_10min  15324 non-null  object 
 6   walking_15min  15324 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.3+ MB


In [ ]:
# merge live_listings with isochrones_combined on property_id
cleaned_listings_sampled = cleaned_listings_sampled.merge(isochrones, on="property_id", how="left")

# impute the isochrones with the nearest neighbour values
cleaned_listings_sampled[['driving_5min_imputed', 'driving_10min_imputed', 'driving_15min_imputed']] = preprocessor.impute_by_nearest_point(cleaned_listings_sampled, ['driving_5min', 'driving_10min', 'driving_15min'])
cleaned_listings_sampled[['walking_5min_imputed', 'walking_10min_imputed', 'walking_15min_imputed']] = preprocessor.impute_by_nearest_point(cleaned_listings_sampled, ['walking_5min', 'walking_10min', 'walking_15min'])

cleaned_listings_sampled.info()




Imputing null values in 3 column(s): ['driving_5min', 'driving_10min', 'driving_15min']
Total rows with null values to impute: 2191
Successfully imputed: 2191 rows, Could not impute (no data available): 0
Imputing null values in 3 column(s): ['walking_5min', 'walking_10min', 'walking_15min']
Total rows with null values to impute: 3690
Successfully imputed: 3690 rows, Could not impute (no data available): 0
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 14497 entries, 0 to 14496
Columns: 289 entries, property_id to walking_15min_imputed
dtypes: Int64(1), float64(265), geometry(1), int64(5), object(17)
memory usage: 32.0+ MB


In [44]:
# save final cleaned_listings_sampled to data/curated/rent_features/cleaned_listings_sampled.csv
cleaned_listings_sampled.to_csv("../data/curated/rent_features/cleaned_listings_sampled.csv", index=False)


In [46]:
# save cleaned_listings_sampled with pickle 
cleaned_listings_sampled.to_pickle("../data/curated/rent_features/cleaned_listings_sampled.pkl")

# Save as Parquet with Spark (ensuring files < 50MB)
print("\nSaving to Parquet format with Spark...")

# Convert geometry column to WKT string format (Parquet doesn't support geometry natively)
cleaned_listings_spark_df = cleaned_listings_sampled.copy()
cleaned_listings_spark_df['coordinates_wkt'] = cleaned_listings_spark_df['coordinates'].apply(lambda geom: geom.wkt)

# Drop the original geometry column and reset to regular DataFrame
cleaned_listings_spark_df = cleaned_listings_spark_df.drop(columns=['coordinates'])
cleaned_listings_spark_df = pd.DataFrame(cleaned_listings_spark_df)

# Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(cleaned_listings_spark_df)

# Estimate the number of partitions needed to keep files under 50MB
# Average row size in memory: 32MB / 14497 rows ≈ 2.2KB per row
# For parquet with snappy compression (typically 50-70% compression ratio)
# Target: ~40MB per file to have buffer, with 2.2KB/row * 0.6 compression ≈ 1.3KB/row
# Records per file: 40MB / 1.3KB ≈ 30,000 records
records_per_partition = 5000  # Conservative estimate to ensure < 50MB per file

num_partitions = max(1, (spark_df.count() + records_per_partition - 1) // records_per_partition)

print(f"Total records: {spark_df.count()}")
print(f"Creating {num_partitions} partitions")
print(f"Estimated records per file: ~{spark_df.count() // num_partitions}")

# Repartition and save
output_path = "../data/curated/rent_features/cleaned_listings_sampled_parquet"

spark_df.repartition(num_partitions) \
    .write \
    .mode("overwrite") \
    .option("compression", "snappy") \
    .option("maxRecordsPerFile", records_per_partition) \
    .parquet(output_path)

print(f"\nParquet files saved to: {output_path}")

# Verify file sizes
import os
parquet_files = [f for f in os.listdir(output_path) if f.endswith('.parquet')]
print(f"\nCreated {len(parquet_files)} parquet file(s):")
for f in sorted(parquet_files):
    size_mb = os.path.getsize(os.path.join(output_path, f)) / (1024 * 1024)
    print(f"  {f}: {size_mb:.2f} MB")


Saving to Parquet format with Spark...


25/10/10 21:24:51 WARN TaskSetManager: Stage 28 contains a task of very large size (263205 KiB). The maximum recommended task size is 1000 KiB.
25/10/10 21:25:12 WARN TaskSetManager: Stage 31 contains a task of very large size (263205 KiB). The maximum recommended task size is 1000 KiB.


Total records: 14497
Creating 3 partitions


25/10/10 21:25:25 WARN TaskSetManager: Stage 34 contains a task of very large size (263205 KiB). The maximum recommended task size is 1000 KiB.


Estimated records per file: ~4832


25/10/10 21:25:41 WARN TaskSetManager: Stage 37 contains a task of very large size (263205 KiB). The maximum recommended task size is 1000 KiB.



Parquet files saved to: ../data/curated/rent_features/cleaned_listings_sampled_parquet

Created 3 parquet file(s):
  part-00000-a0d13ba5-b6f1-47c1-8d12-1799f28da44c-c000.snappy.parquet: 62.23 MB
  part-00001-a0d13ba5-b6f1-47c1-8d12-1799f28da44c-c000.snappy.parquet: 63.06 MB
  part-00002-a0d13ba5-b6f1-47c1-8d12-1799f28da44c-c000.snappy.parquet: 63.01 MB


In [47]:
# Example: How to read the parquet files back into a GeoDataFrame
# Uncomment to test reading:

# Read with Spark
spark_df_loaded = spark.read.parquet("../data/curated/rent_features/cleaned_listings_sampled_parquet")
df_loaded = spark_df_loaded.toPandas()

# Convert WKT back to geometry
from shapely import wkt
df_loaded['coordinates'] = df_loaded['coordinates_wkt'].apply(wkt.loads)
df_loaded = df_loaded.drop(columns=['coordinates_wkt'])

# Convert back to GeoDataFrame
gdf_loaded = gpd.GeoDataFrame(df_loaded, geometry='coordinates')
print(gdf_loaded.info())


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 14497 entries, 0 to 14496
Columns: 286 entries, bathrooms to coordinates
dtypes: float64(263), geometry(1), int64(6), object(16)
memory usage: 31.6+ MB
None


In [48]:
# sort columns alphabetically
gdf_loaded = gdf_loaded.reindex(sorted(gdf_loaded.columns), axis=1)

In [49]:
# print the number of missing values in each column
print(gdf_loaded.isnull().sum())




bathrooms                                           0
bedrooms                                            0
car_spaces                                          0
closest_ptv_station_id                          10754
closest_ptv_station_id_imputed                      0
coordinates                                         0
count_atm                                       12561
count_atm_imputed                                   0
count_bank                                      12326
count_bank_imputed                                  0
count_bar                                       12937
count_bar_imputed                                   0
count_biergarten                                14430
count_biergarten_imputed                            0
count_brothel                                   13811
count_brothel_imputed                               0
count_bus_station                               13517
count_bus_station;shelter                       14484
count_bus_station;shelter_im